เราจะจำแนก รูปภาพแบบหลายหมวดหมู่ โดยใช้ภาพถ่ายจากดาวเทียบ


# 0. Magic Commands


In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# 1. Import Library

In [ ]:
from fastai import *
from fastai.vision import *
from fastai.metrics import accuracy

# 2. ข้อมูล

Download จาก Kaggle


In [ ]:

# ! {sys.executable} -m pip install kaggle --upgrade

* สมัคร Kaggle
* เข้าหน้า My Account
* ในหมวด API กดปุ่ม <kbd>Create New API Token</kbd>
* Download ไฟล์ kaggle.json
* เปิด Jupyter Notebook Dashboard ไปอัพโหลดไฟล์ไว้ใน path ปัจจุบัน




In [ ]:
# ! mkdir -p ~/.kaggle/
# ! mv kaggle.json ~/.kaggle/


In [ ]:
path = Config.data_path()/'planet'
path.mkdir(parents=True, exist_ok=True)
path

1. เปิดหน้าเว็บการแข่งขัน Kaggle [Planet: Understanding the Amazon from Space Competition](https://www.kaggle.com/c/planet-understanding-the-amazon-from-space)
2. เข้าไปกด Join การแข่งขัน กด Accept กติกา 
3. Uncomment แล้ว Run โค้ดด้านล่างเพื่อ Download ชุดข้อมูล

In [ ]:

# ! kaggle competitions download -c planet-understanding-the-amazon-from-space -f train-jpg.tar.7z -p {path}  
# ! kaggle competitions download -c planet-understanding-the-amazon-from-space -f train_v2.csv -p {path}  
# ! unzip -q -n {path}/train_v2.csv.zip -d {path}

Install โปรแกรม 7zip สำหรับแตกไฟล์ นามสกุล .7z

In [ ]:
# ! conda install --yes --prefix {sys.prefix} -c haasad eidl7zip

แตกไฟล์รูป train-jpg.tar.7z

In [ ]:
# ! 7za -bd -y -so x {path}/train-jpg.tar.7z | tar xf - -C {path.as_posix()}

# 3. เตรียมข้อมูล

กกกก

In [ ]:

df = pd.read_csv(path/'train_v2.csv')
df.head()

In [ ]:
tfms = get_transforms(flip_vert=True, max_lighting=0.1, max_zoom=1.05, max_warp=0.)


In [ ]:
np.random.seed(42)
datasource = (ImageList.from_csv(path, 'train_v2.csv', folder='train-jpg', suffix='.jpg')
       .split_by_rand_pct(0.2)
       .label_from_df(label_delim=' '))

size = 128 x 128 Pixel

In [ ]:
databunch = (datasource.transform(tfms, size=128)
        .databunch().normalize(imagenet_stats))

## สำรวจข้อมูล ##

ลองดูข้อมูล Batch แรก ด้วย show_batch สั่งให้แสดง รูป พร้อม label

*เราสามารถรัน cell นี้หลายครั้ง เพื่อเรียกดู batch ต่อ ๆ ไป ได้เรื่อย ๆ เป็นการสำรวจข้อมูล*

In [ ]:
databunch.show_batch(rows=3, figsize=(12,9))

 รูปหนึ่งรูปสามารถเป็น จำแนกได้เป็น ถนน พื้นที่เพาะปลูก มีบังบางส่วน

In [ ]:
databunch.classes, databunch.c, len(databunch.train_ds), len(databunch.valid_ds)

# 4. สร้างโมเดล



F beta  และ accuracy_thresh 0.2


In [ ]:
acc_02 = partial(accuracy_thresh, thresh=0.2)
f_score = partial(fbeta, thresh=0.2)

In [ ]:
learner = cnn_learner(databunch, models.resnet50, metrics=[acc_02, f_score]).to_fp16()

# 5. เริ่มต้นเทรนโมเดล

เทรนด้วยค่า Default ไป 8 Epoch

In [ ]:
learner.fit_one_cycle(8)

เรามา Save Model ที่เราเพิ่งเทรนไปเก็บไว้ก่อน

In [ ]:
learner.save('03a-resnet50-1')

โหลดโมเดลที่เรา Save ไว้ ขึ้นมาใหม่ เตรียมเทรนในขึ้นตอนถัดไป (ที่คอมเม้นท์ไว้ เพราะถ้าไม่ได้เปลี่ยนอะไรก็จะได้ไม่ต้องรัน)

In [ ]:
# learner.load('03a-resnet50-1')

# 6. เทรนต่อ

unfreeze layer ทุก Layer ให้สามารถเทรนได้ แล้วเทรนต่อทั้งโมเดล

In [ ]:
learner.unfreeze()
learner.lr_find()
learner.recorder.plot()

In [ ]:
learner.fit_one_cycle(6, max_lr=slice(3e-6,3e-3))

In [ ]:
learner.save('03a-resnet50-2')

In [ ]:
# learner.load('03a-resnet50-2')

# 7. เทรนต่อ 2

Transfer Learning โมเดลที่เราเพิ่งเทรนเสร็จ นำมาเทรนต่อด้วย Data ทีรูปใหญ่ขึ้น เป็น 256 x 256 Pixel

In [ ]:
databunch = (src.transform(tfms, size=256)
        .databunch().normalize(imagenet_stats))

learner.data = databunch
databunch.train_ds[0][0].shape

Freeze ใหม่ เพื่อเทรนเฉพาะ Layer สุดท้าย

In [ ]:
learner.freeze()

In [ ]:
learn.fit_one_cycle(5, 3e-3)

In [ ]:
learn.save('03a-resnet50-256-1')

In [ ]:
# learn.load('03a-resnet50-256-1')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-6, 3e-4))

## สำเร็จแล้ว ##

ฟฟฟฟฟ


เรามา Save Model ที่เราเพิ่งเทรนไปเก็บไว้ก่อน

In [ ]:
learner.save('03a-resnet50-256-2')

In [ ]:
# learner.load('03a-resnet50-256-2')

In [ ]:
learner.export()

# 7. ดูผลลัพธ์

In [ ]:
interpretation = ClassificationInterpretation.from_learner(learner)

## โมเดลจำแนกผิวได้ถูกต้อง

สั่งให้ plot_top_losses คือ แสดง record ที่ ค่า loss น้อยที่สุด หรือพูดง่าย ๆ ว่าแสดงรายการที่โมเดลทายถูกต้อง 9 อันดับแรก โดยมีการแสดง heat map สีแดง ให้ดูด้วยว่า model พิจารณาเน้นจากส่วนไหนของรูปเป็นหลัก เรียกว่า Attention ที่ไว้เราจะอธิบายต่อไป

In [ ]:
interpretation.plot_top_losses(9, figsize=(10,10), largest=False)

จะเห็นได้ว่าโมเดลพิจารณาถูกจุดจริง ๆ โฟกัสตรงตำแหน่ง

## โมเดลจำแนก ผิดหมวด

สั่งให้ plot_top_losses คือ แสดง record ที่ ค่า loss มากที่สุด หรือพูดง่าย ๆ ว่าแสดงรายการที่โมเดลทายผิด 9 อันดับแรก 

In [ ]:
interpretation.plot_top_losses(9, figsize=(10,10), largest=True)

จะเห็นได้ว่า...

# 8. ส่งประกวด

In [ ]:
#! kaggle competitions download -c planet-understanding-the-amazon-from-space -f test-jpg.tar.7z -p {path}  
#! 7za -bd -y -so x {path}/test-jpg.tar.7z | tar xf - -C {path}
#! kaggle competitions download -c planet-understanding-the-amazon-from-space -f test-jpg-additional.tar.7z -p {path}  
#! 7za -bd -y -so x {path}/test-jpg-additional.tar.7z | tar xf - -C {path}

In [ ]:
test = ImageList.from_folder(path/'test-jpg').add(ImageList.from_folder(path/'test-jpg-additional'))
len(test)

In [ ]:
learner = load_learner(path, test=test)
preds, _ = learner.get_preds(ds_type=DatasetType.Test)

In [ ]:
thresh = 0.2
labelled_preds = [' '.join([learn.data.classes[i] for i,p in enumerate(pred) if p > thresh]) for pred in preds]

In [ ]:
labelled_preds[:5]

In [ ]:
fnames = [f.name[:-4] for f in learn.data.test_ds.items]
df = pd.DataFrame({'image_name':fnames, 'tags':labelled_preds}, columns=['image_name', 'tags'])
df.to_csv(path/'submission.csv', index=False)

In [ ]:
# ! kaggle competitions submit planet-understanding-the-amazon-from-space -f {path/'submission.csv'} -m "My submission"

# 9. สรุป


# Credit 

* [FastAI: Practical Deep Learning for Coders, v3 - Lesson 3](https://course.fast.ai/videos/?lesson=3)